In [32]:
import pandas as pd
from sklearn.metrics import f1_score

from lbl2vec import Lbl2Vec, Lbl2TransformerVec

from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import strip_tags

In [78]:
def evaluate_lbl2vec_on_dataset(dataset_name: str, n_iterations: int = 1, n_words: int = 1):
    def tokenize(doc):
        return simple_preprocess(strip_tags(doc), deacc=True, min_len=2, max_len=15)
    
    # -- Documents
    documents_df = pd.read_csv(f'../datasets/data/{dataset_name}/documents.csv')
    documents_df['tagged_docs'] = documents_df.apply(
        lambda row: TaggedDocument(tokenize(row['document']), [str(row.name)]), axis=1
    )
    documents_df['doc_key'] = documents_df.index.astype(str)

    # -- Labels
    labels_df = pd.read_csv(f'../datasets/data/{dataset_name}/labels.csv')
    labels_df['number_of_keywords'] = labels_df['class_keywords'].apply(lambda keywords: len(keywords))

    keywords_cached_df = pd.read_csv(f'cache/{dataset_name}_spacy_keywords.csv')
    keywords_cached_df['class_result_keywords'] = keywords_cached_df['class_result_keywords'].apply(lambda x: x[1:-1].replace("'", '').split(', '))
    similar_keywords = dict(zip(keywords_cached_df['class_raw_keywords'], keywords_cached_df['class_result_keywords']))
    labels_df['class_result_keywords'] = labels_df.apply(
        lambda row: similar_keywords[row['class_keywords']][:n_words], axis=1
    )
    documents_df['class_keywords'] = documents_df['class_name'].apply(lambda class_name: labels_df[labels_df['class_name'] == class_name]['class_keywords'].values[0])

    # -- Evaluation
    avg_f1_score_train = 0
    avg_f1_score_test = 0

    for i in range(n_iterations):
        print(f'[INFO] {dataset_name} | Iteration {i+1}/{n_iterations}...' + ' ' * 20, end='\r')

        # -- Model
        lbl2vec_model = Lbl2Vec(
            keywords_list=[item for item in labels_df['class_result_keywords']], 
            tagged_documents=list(documents_df['tagged_docs'][documents_df['dataset_type'] == 'train']), 
            label_names=[item for item in labels_df['class_keywords']], 
            min_count=2,
            verbose=False
        )
        lbl2vec_model.fit()

        train_docs_lbl_similarities = lbl2vec_model.predict_model_docs()
        test_docs_lbl_similarities = lbl2vec_model.predict_new_docs(tagged_docs=documents_df['tagged_docs'][documents_df['dataset_type'] == 'test'])

        evaluation_train = train_docs_lbl_similarities.merge(documents_df[documents_df['dataset_type']=='train'], left_on='doc_key', right_on='doc_key')
        evaluation_test = test_docs_lbl_similarities.merge(documents_df[documents_df['dataset_type']=='test'], left_on='doc_key', right_on='doc_key')

        y_true_train = evaluation_train['class_keywords']
        y_pred_train = evaluation_train['most_similar_label']

        y_true_test = evaluation_test['class_keywords']
        y_pred_test = evaluation_test['most_similar_label']

        current_f1_score_train = f1_score(y_true_train, y_pred_train, average='micro')
        current_f1_score_test = f1_score(y_true_test, y_pred_test, average='micro')

        avg_f1_score_train += current_f1_score_train
        avg_f1_score_test += current_f1_score_test

        print(f'[INFO] Iteration {i+1}/{n_iterations} - Train F1 Score: {current_f1_score_train:.4f} - Test F1 Score: {current_f1_score_test:.4f}' + ' ' * 10, end='\r')

    avg_f1_score_train /= n_iterations
    avg_f1_score_test /= n_iterations

    return avg_f1_score_train, avg_f1_score_test

In [82]:
evaluate_lbl2vec_on_dataset('BBC_News', n_iterations=10, n_words=10)

(0.7860674157303371, 0.7714606741573033)